In [1]:
from pathlib import Path

from src.inspector_git.linker.models import GitProject
from src.inspector_git.linker.transformers import GitProjectTransformer
from src.inspector_git.reader.iglog.readers.ig_log_reader import IGLogReader
from src.run_blame import check_blame
import sys
# sys.setrecursionlimit(16336)

def build_and_check_project(iglog_path: str, repo_path: str, compute_annotated_lines :bool) -> GitProject:
    iglog_file = Path(iglog_path)

    with open(iglog_file, "r", encoding="utf-8") as f:
        git_log_dto = IGLogReader().read(f)

    transformer = GitProjectTransformer(
        git_log_dto,
        name=iglog_file.stem,
        compute_annotated_lines=compute_annotated_lines,
    )
    project = transformer.transform()

    if compute_annotated_lines:
        check_blame(project, repo_path)

    return project

project = build_and_check_project(
    "../../test-input/inspector-git/zeppelin.iglog",
    "/home/vortex/Work/BachelorThesis/voyager-target/test_inspector_git_repo/TestInspectorGitRepo",
    False
)

2025-09-22 11:21:38,540 [WARNING] src.inspector_git.linker.transformers: Found change link to file for deletion outside the file changes list: b89ba779ad94ec256bd8ca52bf1183c18f940ff8-zeppelin-zengine/src/test/java/com/nflabs/zeppelin/zengine/ExecStatementTest.java->dev/null
2025-09-22 11:21:38,629 [WARNING] src.inspector_git.linker.transformers: Found change link to file for deletion outside the file changes list: 017896fc7d2e7ad466ceba071bfbca22b4b9af20-zeppelin-zengine/src/test/java/com/nflabs/zeppelin/zengine/ExecStatementTest.java->dev/null
2025-09-22 11:21:39,641 [WARNING] src.inspector_git.linker.transformers: Found change link to file for deletion outside the file changes list: 022b79ce1e7f0af6ec7fd27f34cdf80cbbe8234b-zeppelin-zengine/src/main/java/com/nflabs/zeppelin/zengine/Z.java->dev/null
2025-09-22 11:21:39,645 [WARNING] src.inspector_git.linker.transformers: Found change link to file for deletion outside the file changes list: 022b79ce1e7f0af6ec7fd27f34cdf80cbbe8234b-zepp

In [2]:
count = 0
for change in project.change_registry.all:
    file = project.file_registry.get_by_id(change.file.id)
    if not file:
        print(count)
        count += 1

In [3]:
import pickle

with open("project.pkl", "wb") as f:
    pickle.dump(project, f)

print("GitProject saved to project.pkl")

GitProject saved to project.pkl


In [4]:
with open("project.pkl", "rb") as f:
    loaded_project = pickle.load(f)

In [5]:
loaded_project == project

True